###Goal
  - To predict the amount of property damage that can be expected by tornadoes based on the Fujita scale types.

###Import Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

%matplotlib inline

In [ ]:
import sagemaker 
import boto3
from sagemaker.session import s3_input, Session

###Create S3 Buckets

In [ ]:
bucket = 'awsdisasterresponsehackathon'
bucket_region = boto3.session.Session().region_name
print(bucket_region)

In [ ]:
s3 = boto3.resource('s3')
try: 
    if  bucket_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket)
    print('S3 bucket successfully created')
except Exception as e:
    print('S3 error: ',e)

In [ ]:
#setting output path for the trained model

prefix = 'linear-regression-model'
output_path = 's3://{}/{}/output'.format(bucket, prefix)
print(output_path)

In [ ]:
bucket = 'awsdisasterresponsehackathon'
data_key = '1950-2019_actual_tornadoes.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

raw_tornado_catalog = pd.read_csv(data_location)

###Feature Engineering

In [ ]:
#location of data
https://www.spc.noaa.gov/wcm/

In [ ]:
#description of data
https://www.spc.noaa.gov/wcm/data/SPC_severe_database_description.pdf

In [ ]:
#feature selection
tornado_catalog = raw_tornado_catalog[['mag', 'loss', 'st']]

In [ ]:
#one hot encoding
tornado_catalog_encoded = pd.get_dummies(tornado_catalog)

In [ ]:
tornado_catalog_encoded

###Import/Read Data

In [ ]:
#import the csv data
#the dataset is from the NOAA Storm Prediction Center

#copy path to csv file
raw_tornado_catalog = pd.read_csv('') 

In [ ]:
#feature selection
tornado_catalog = raw_tornado_catalog[['mag', 'loss', 'st']]

In [ ]:
#one hot encoding
tornado_catalog_encoded = pd.get_dummies(tornado_catalog)

In [ ]:
tornado_catalog_encoded

,mag,loss,st_AK,st_AL,st_AR,st_AZ,st_CA,st_CO,st_CT,st_DC,st_DE,st_FL,st_GA,st_HI,st_IA,st_ID,st_IL,st_IN,st_KS,st_KY,st_LA,st_MA,st_MD,st_ME,st_MI,st_MN,st_MO,st_MS,st_MT,st_NC,st_ND,st_NE,st_NH,st_NJ,st_NM,st_NV,st_NY,st_OH,st_OK,st_OR,st_PA,st_PR,st_RI,st_SC,st_SD,st_TN,st_TX,st_UT,st_VA,st_VT,st_WA,st_WI,st_WV,st_WY
0,3,6.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3,5.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,4.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,3.00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2,5.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60109,1,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60110,2,0.05,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60111,1,0.10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60112,1,0.01,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
tornado_catalog_encoded.dtypes

mag        int64
loss     float64
st_AK      uint8
st_AL      uint8
st_AR      uint8
st_AZ      uint8
st_CA      uint8
st_CO      uint8
st_CT      uint8
st_DC      uint8
st_DE      uint8
st_FL      uint8
st_GA      uint8
st_HI      uint8
st_IA      uint8
st_ID      uint8
st_IL      uint8
st_IN      uint8
st_KS      uint8
st_KY      uint8
st_LA      uint8
st_MA      uint8
st_MD      uint8
st_ME      uint8
st_MI      uint8
st_MN      uint8
st_MO      uint8
st_MS      uint8
st_MT      uint8
st_NC      uint8
st_ND      uint8
st_NE      uint8
st_NH      uint8
st_NJ      uint8
st_NM      uint8
st_NV      uint8
st_NY      uint8
st_OH      uint8
st_OK      uint8
st_OR      uint8
st_PA      uint8
st_PR      uint8
st_RI      uint8
st_SC      uint8
st_SD      uint8
st_TN      uint8
st_TX      uint8
st_UT      uint8
st_VA      uint8
st_VT      uint8
st_WA      uint8
st_WI      uint8
st_WV      uint8
st_WY      uint8
dtype: object

In [ ]:
#index and aggregate the data by state
#providing a count of the number of tornadoes and a sum of the total property damage

tornado_catalog_encoded = pd.DataFrame
({'count': tornado_catalog_encoded.groupby(['st'])['loss'].count(), 'total_loss': tornado_catalog_encoded.groupby(['st'])['loss'].sum(), 'f-scale': tornado_catalog_encoded.groupby(['mag'])['loss'].count()})


###Create Training and Test Data

In [ ]:
X = tornado_catalog_encoded.drop("mag", axis=1)
y = tornado_catalog_encoded["mag"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)
len(X), len(X_train), len(X_test)

###Model

In [ ]:
tf.random.set_seed(33)

#create a model

tornado_model_A = tf.keras.Sequential([
    tf.keras.layers.Dense(400, activation="relu"),
    tf.keras.layers.Dense(400, activation="relu"),
    tf.keras.layers.Dense(400, activation="relu"),
    tf.keras.layers.Dense(400, activation="relu"),
    tf.keras.layers.Dense(40, activation="relu"),
    tf.keras.layers.Dense(1)
])

#compile the model

tornado_model_A.compile(loss=tf.keras.losses.mae,
                        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                        metrics=["mae"])

#fit the model
history_A = tornado_model_A.fit(X_train, y_train, epochs=500)

In [ ]:
tornado_model_A.summary()

###Evaluation

In [ ]:
tornado_model_A.evaluate(X_test, y_test)

In [ ]:
y_train.median(), y_train.mean()

In [ ]:
pd.DataFrame(history_A.history).plot()
plt.ylabel("loss")
plt.xlabel("epochs")